In [1]:
from heapq import merge
from operator import index
from sys import displayhook
import mysql.connector
from mysql.connector import Error
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import mysql.connector as sql
import pandas as pd
from IPython.display import display

# 1. Pilih Database
connection = mysql.connector.connect(host='localhost',
                                         database='cargo_route', # masukan nama database di kiri
                                         user='root',
                                         password='')
cursor = connection.cursor(dictionary=True)

In [2]:
#2. masukkan querry 
#a. querry untuk table sample (sample rute dengan asal dan tujuan berupa int/kode)
samplequery = "SELECT * FROM sample;" 
masterquery = "SELECT * FROM city;"

In [3]:
#3. map sebelum di scale 
maps = pd.read_sql(samplequery,connection)
route = pd.read_sql(samplequery,connection)
display(maps)

C:\Users\LENOVO\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\LENOVO\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,sample_id,origin,destination,distance
0,1,1,3,430
1,2,1,5,537
2,3,1,4,407
3,4,1,10,909
4,5,1,11,1259
5,6,1,9,917
6,7,1,14,1397
7,8,1,7,671
8,9,1,6,808
9,10,3,11,1025


In [4]:
city = pd.read_sql(masterquery,connection)
display(city)

C:\Users\LENOVO\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,city_id,city_name
0,1,Jakarta
1,2,Bandung
2,3,Yogyakarta
3,4,Semarang
4,5,Solo
5,6,Surabaya
6,7,Malang
7,8,Denpasar
8,9,Banjarmasin
9,10,Palangkaraya


In [5]:
#map dengan asal dan tujuan string bukan int (hanya untuk cek/tampilkan daftar rute)
def map_transformed(origin,destination):
    for kodeC in maps[maps.columns[1]].values: # ambil kode primary yang ada di kolom origin
        for k,v in city.values: # ambil PK dan nama kota yang ada di table kota
            if kodeC == k: # jika PK di kolom origin yang sedang di baca sama dengan salah satu Pk di table kota,
                origin.append(v) # tambahkan nama kotanya.
    route[route.columns[1]] = origin # ubah kolom origin di dataframe berdasarkan list penampung
# untuk destination kurang lebih sama
    for kodeC in maps[maps.columns[2]].values:
        for k,v in city.values:
            if kodeC == k:
                destination.append(v)
    route[route.columns[2]] = destination
    display(route)
origin = []  #list menampung transformasi primary code di kolom origin
destination = [] #list menampung transformasi primary code di kolom destination
map_transformed(origin,destination)

,sample_id,origin,destination,distance
0,1,Jakarta,Yogyakarta,430
1,2,Jakarta,Solo,537
2,3,Jakarta,Semarang,407
3,4,Jakarta,Palangkaraya,909
4,5,Jakarta,Balikpapan,1259
5,6,Jakarta,Banjarmasin,917
6,7,Jakarta,Makassar,1397
7,8,Jakarta,Malang,671
8,9,Jakarta,Surabaya,808
9,10,Yogyakarta,Balikpapan,1025


In [6]:
def graph_maker(graph):
    for row in maps.values:
        origin_db = row[1]
        destination_db = row[2]
        distance_db = row[3]

        if origin_db in graph:
            graph[origin_db][destination_db] = distance_db
        else:
            graph[origin_db]={}
            graph[origin_db][destination_db] = distance_db
    return print(graph)
graph = {}
graph_maker(graph)


{1: {3: 430, 5: 537, 4: 407, 10: 909, 11: 1259, 9: 917, 14: 1397, 7: 671, 6: 808}, 3: {11: 1025}, 4: {6: 257}, 6: {2: 776, 8: 303, 14: 791, 17: 1674, 9: 484}, 9: {11: 339}, 11: {12: 517, 13: 463, 14: 515}, 13: {14: 463}, 14: {18: 959, 15: 439, 16: 735, 17: 1233}}


In [7]:
def graph_transformed(route_graph):
    for row in route.values:
        origin_db = row[1]
        destination_db = row[2]
        distance_db = row[3]

        if origin_db in route_graph:
            route_graph[origin_db][destination_db] = distance_db
        else:
            route_graph[origin_db]={}
            route_graph[origin_db][destination_db] = distance_db
    return print(route_graph)
route_graph = {}
graph_transformed(route_graph)


{'Jakarta': {'Yogyakarta': 430, 'Solo': 537, 'Semarang': 407, 'Palangkaraya': 909, 'Balikpapan': 1259, 'Banjarmasin': 917, 'Makassar': 1397, 'Malang': 671, 'Surabaya': 808}, 'Yogyakarta': {'Balikpapan': 1025}, 'Semarang': {'Surabaya': 257}, 'Surabaya': {'Bandung': 776, 'Denpasar': 303, 'Makassar': 791, 'Manado': 1674, 'Banjarmasin': 484}, 'Banjarmasin': {'Balikpapan': 339}, 'Balikpapan': {'Tarakan': 517, 'Palu': 463, 'Makassar': 515}, 'Palu': {'Makassar': 463}, 'Makassar': {'Ambon': 959, 'Kendari': 439, 'Gorontalo': 735, 'Manado': 1233}}


In [197]:
def Unpacking_Track(Close_list):    
    List_lenght = len(Close_list)
    Result = []
    Child_list = {}
    Result.append(Close_list[-1])
    for l in range (List_lenght):
        Current_node = Close_list[-1]
        print("Current Node : ",Current_node)
        print(Close_list)
        if Current_node in Maps:
            Child_list=(Maps[Current_node])
            print("Current Child List : ",Child_list)
            if Result[-1] in Child_list:  
                Result.append(Current_node)
                Close_list.remove(Current_node)
                print('Result :',Result)   

            else:
                Close_list.remove(Current_node)
                print('Result :',Result) 

        else:
            Close_list.remove(Current_node)
            print('Result :',Result) 
            
    Actual_path = [] 
 
    # Substract one from the length of the original array to start from the last index: 
    reducer = len(Result)-1 

    # Reverse the list inside a for loop: 
    for i in Result:
     Actual_path.append(Result[reducer]) # Append the result to the empty list 
     reducer -=1 # Decrease the index by one at each iteration using the reducer
    
    return Actual_path
    
    



In [135]:
def Sorting_Path(Result):
    Actual_path = [] 
 
    # Substract one from the length of the original array to start from the last index: 
    reducer = len(Result)-1 

    # Reverse the list inside a for loop: 
    for i in Result:
     Actual_path.append(Result[reducer]) # Append the result to the empty list 
     reducer -=1 # Decrease the index by one at each iteration using the reducer
    return Actual_path



In [179]:

def Best_First_Search(Maps,Asal,Tujuan):

    Open_list = {}
    Close_list = []

    Current_node = Asal
    if Current_node in Maps:
        Open_list.update(Maps[Current_node])
    X = len(Open_list)

    while Tujuan not in Close_list:
        print(" Current Node: (",Current_node,")")

        if Current_node in Maps and Current_node not in Close_list:
            Open_list.update(Maps[Current_node])
            if Current_node in Open_list:
                del Open_list[Current_node]
            X = len(Open_list)
            if X == 0:
                Close_list = ['No More Route']
                break
            print(" --Open List :",Open_list)
            if Tujuan not in Open_list:
                Close_list.append(Current_node)
                Current_node = min(Open_list, key=Open_list.get)
            elif Tujuan in Open_list:
                for Node, Jarak in Open_list.items():
                    if Node == Tujuan:
                        Close_list.append(Current_node)
                        Current_node = Node
                        print("Close_list",Close_list)
                        break
        else:
            del Open_list[Current_node]
            X = len(Open_list)
            print(" --Open List :",Open_list)
            if X == 0:
                Close_list = ['No More Route']
                break
            if Tujuan not in Open_list:
                Close_list.append(Current_node)
                Current_node = min(Open_list, key=Open_list.get)
            elif Tujuan in Open_list:
                for node, jarak in Open_list.items():
                    if node == Tujuan:
                        Close_list.append(Current_node)
                        Current_node = Node
                        print("Close_list",Close_list)
                        break
    return Unpacking_Track(Close_list)      

    

In [204]:
def Path_lenght(Actual_path):   
    Curr_node = Actual_path[0]
    Path_lenght = 0
    while Curr_node != Actual_path[-1]:
    # for z in range(25):
        print(Curr_node)
        print (Maps[Curr_node])
        for k in a:
            if k in Maps[Curr_node]:
                Path_lenght = Path_lenght + Maps[Curr_node][k]
                Curr_node = k
                print(Path_lenght)
                break
    return Actual_path,Path_lenght

In [177]:
Close_list=['Jakarta', 'Semarang', 'Surabaya', 'Bandung']
List_lenght = len(Close_list)
Result = []
Child_list = {}
Result.append(Close_list[-1])
for l in range (List_lenght):
    Current_node = Close_list[-1]
    print("Current Node : ",Current_node)
    print(Close_list)
    if Current_node in Maps:
        Child_list=(Maps[Current_node])
        print("Current Child List : ",Child_list)
        if Result[-1] in Child_list:  
            Result.append(Current_node)
            Close_list.remove(Current_node)
            print('Result :',Result)   
        
        else:
            Close_list.remove(Current_node)
            print('Result :',Result) 
            
    else:
        Close_list.remove(Current_node)
        print('Result :',Result) 
      


Current Node :  Bandung
['Jakarta', 'Semarang', 'Surabaya', 'Bandung']
Result : ['Bandung']
Current Node :  Surabaya
['Jakarta', 'Semarang', 'Surabaya']
Current Child List :  {'Bandung': 776, 'Denpasar': 303, 'Makassar': 791, 'Manado': 1674, 'Banjarmasin': 484}
Result : ['Bandung', 'Surabaya']
Current Node :  Semarang
['Jakarta', 'Semarang']
Current Child List :  {'Surabaya': 257}
Result : ['Bandung', 'Surabaya', 'Semarang']
Current Node :  Jakarta
['Jakarta']
Current Child List :  {'Yogyakarta': 430, 'Solo': 537, 'Semarang': 407, 'Palangkaraya': 909, 'Balikpapan': 1259, 'Banjarmasin': 917, 'Makassar': 1397, 'Malang': 671, 'Surabaya': 808}
Result : ['Bandung', 'Surabaya', 'Semarang', 'Jakarta']


In [209]:
a= Best_First_Search(route_graph,'Jakarta','Gorontalo')
Actual_route=Path_lenght(a)
print("Jalur :",Actual_route)

    


 Current Node: ( Jakarta )
 --Open List : {'Yogyakarta': 430, 'Solo': 537, 'Semarang': 407, 'Palangkaraya': 909, 'Balikpapan': 1259, 'Banjarmasin': 917, 'Makassar': 1397, 'Malang': 671, 'Surabaya': 808}
 Current Node: ( Semarang )
 --Open List : {'Yogyakarta': 430, 'Solo': 537, 'Palangkaraya': 909, 'Balikpapan': 1259, 'Banjarmasin': 917, 'Makassar': 1397, 'Malang': 671, 'Surabaya': 257}
 Current Node: ( Surabaya )
 --Open List : {'Yogyakarta': 430, 'Solo': 537, 'Palangkaraya': 909, 'Balikpapan': 1259, 'Banjarmasin': 484, 'Makassar': 791, 'Malang': 671, 'Bandung': 776, 'Denpasar': 303, 'Manado': 1674}
 Current Node: ( Denpasar )
 --Open List : {'Yogyakarta': 430, 'Solo': 537, 'Palangkaraya': 909, 'Balikpapan': 1259, 'Banjarmasin': 484, 'Makassar': 791, 'Malang': 671, 'Bandung': 776, 'Manado': 1674}
 Current Node: ( Yogyakarta )
 --Open List : {'Solo': 537, 'Palangkaraya': 909, 'Balikpapan': 1025, 'Banjarmasin': 484, 'Makassar': 791, 'Malang': 671, 'Bandung': 776, 'Manado': 1674}
 Curren

In [24]:
# ALGORITMA BEST SEARCH FIRST LAMA**
Asal = 'Jakarta'
Tujuan = 'Ternate'
Maps = route_graph

Open_list = {}
Close_list = []

Current_node = Asal
if Current_node in Maps:
    Open_list.update(Maps[Current_node])
X = len(Open_list)
    
while Tujuan not in Close_list:
    print(" Current Node: (",Current_node,")")

    if Current_node in Maps and Current_node not in Close_list:
        Open_list.update(Maps[Current_node])
        if Current_node in Open_list:
            del Open_list[Current_node]
        X = len(Open_list)
        if X != 0:
            Jarak_terpendek = min(Open_list.values())
        elif X == 0:
            Close_list = ['No More Route']
            break
        print(" --Open List :",Open_list)
        if Tujuan not in Open_list:
            for Node, Jarak in Open_list.items():
                if Jarak == Jarak_terpendek:
                    Close_list.append(Current_node)
                    Current_node = Node
                    print("Close_list",Close_list)

        elif Tujuan in Open_list:
#             print("ERROR 01")
            for Node, Jarak in Open_list.items():
                if Node == Tujuan:
                    Close_list.append(Current_node)
                    Current_node = Node
                    print("Close_list",Close_list)
                    break

    else:
        del Open_list[Current_node]
        X = len(Open_list)
        print(" --Open List :",Open_list)
        if X != 0:
            Jarak_terpendek = min(Open_list.values())
        elif X == 0:
            Close_list = ['No More Route']
            break
        if Tujuan not in Open_list:
            for Node, Jarak in Open_list.items():
                if Jarak == Jarak_terpendek:
                    Close_list.append(Current_node)
                    Current_node = Node
                    print("Close_list",Close_list)
        elif Tujuan in Open_list:
            for node, jarak in Open_list.items():
                if node == Tujuan:
                    Close_list.append(Current_node)
                    Current_node = Node
                    print("Close_list",Close_list)
                    break
                    
#Mencari Jalur yang sebenarnya dari Close_list                    
Result = []
Child_list = {}
Current_node = Close_list[-1]
Result.append(Current_node)
Close_list.remove(Current_node)
print(Close_list)
for l in range(7):
    Current_node = Close_list[-1]
    print(Current_node)
    if Current_node in Maps:
        Child_list=(Maps[Current_node])
        print(Child_list)
        if Result[-1] in Child_list:
            Result.append(Current_node)
            Close_list.remove(Current_node)
            print('Result',Result)            
        else:
            Close_list.remove(Current_node)
            print('Result',Result)            
    else:
        Close_list.remove(Current_node)
        print('Result',Result) 
        
# Create an empty list to hold the reversed array: 
Actual_path = [] 
 
# Substract one from the length of the original array to start from the last index: 
reducer = len(Result)-1 
 
# Reverse the list inside a for loop: 
for i in Result:
 Actual_path.append(Result[reducer]) # Append the result to the empty list 
 reducer -=1 # Decrease the index by one at each iteration using the reducer 
 
print('Jalur :',Actual_path)

 Current Node: ( Jakarta )
 --Open List : {'Yogyakarta': 430, 'Solo': 537, 'Semarang': 407, 'Palangkaraya': 909, 'Balikpapan': 1259, 'Banjarmasin': 917, 'Makassar': 1397, 'Malang': 671, 'Surabaya': 808}
Close_list ['Jakarta']
 Current Node: ( Semarang )
 --Open List : {'Yogyakarta': 430, 'Solo': 537, 'Palangkaraya': 909, 'Balikpapan': 1259, 'Banjarmasin': 917, 'Makassar': 1397, 'Malang': 671, 'Surabaya': 257}
Close_list ['Jakarta', 'Semarang']
 Current Node: ( Surabaya )
 --Open List : {'Yogyakarta': 430, 'Solo': 537, 'Palangkaraya': 909, 'Balikpapan': 1259, 'Banjarmasin': 484, 'Makassar': 791, 'Malang': 671, 'Bandung': 776, 'Denpasar': 303, 'Manado': 1674}
Close_list ['Jakarta', 'Semarang', 'Surabaya']
 Current Node: ( Denpasar )
 --Open List : {'Yogyakarta': 430, 'Solo': 537, 'Palangkaraya': 909, 'Balikpapan': 1259, 'Banjarmasin': 484, 'Makassar': 791, 'Malang': 671, 'Bandung': 776, 'Manado': 1674}
Close_list ['Jakarta', 'Semarang', 'Surabaya', 'Denpasar']
 Current Node: ( Yogyakarta

IndexError: list index out of range